# **LIMPIEZA DE DATOS**



## Configuración inicial

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np

## Carga de base Parquet



In [3]:
# pip install pyarrow

In [4]:
# Cargamos base Parquet y revisamos tipos de variables
PARQUET_PATH = r"C:\Users\Hugo Jazyel\OneDrive\INEI\PEU-CD\Cursos especializados\Proyecto Integrador\enaho_merged.parquet"

df = pd.read_parquet(PARQUET_PATH)

print("✅ Archivo Parquet cargado correctamente.")
print("Filas:", df.shape[0], "| Columnas:", df.shape[1])
print("\nTipos de datos (muestra):")
print(df.dtypes)

✅ Archivo Parquet cargado correctamente.
Filas: 438428 | Columnas: 38

Tipos de datos (muestra):
anio             object
conglome         object
vivienda         object
hogar            object
codperso         object
txt505           object
txt505b          object
txt506           object
p208a              int8
p501            float64
p502            float64
p503            float64
p504            float64
p5041           float64
p5042           float64
p5043           float64
p5044           float64
p5045           float64
p5046           float64
p5047           float64
p5048           float64
p5049           float64
p50410          float64
p50411          float64
p207               int8
p301a           float64
p506r4          float64
p507            float64
p510            float64
p505r4          float64
p207_label       object
p301a_label      object
p506r4_label     object
p507_label       object
p510_label       object
p505r4_label     object
p301a1          float64
p301a1_label   

In [5]:
# Filtramos solo personas con ocupación, ya que solo estos son considerados en la p505
cols_filter = [
    "p501", "p502", "p503", "p5041", "p5042", "p5043", "p5044",
    "p5045", "p5046", "p5047", "p5048", "p5049", "p50410", "p50411"
]

# Aseguramos que existan en la base
cols_exist = [c for c in cols_filter if c in df.columns]

# Filtro: 1 en cualquiera de las columnas y sin NA
mask_valid = df[cols_exist].apply(lambda x: (x == 1) & (~x.isna())).any(axis=1)

# Aplicamos filtro
df = df.loc[mask_valid].copy()

print(f"✅ Filtro aplicado: {df.shape[0]} filas restantes tras excluir NA y mantener == 1.")
df.shape

✅ Filtro aplicado: 316025 filas restantes tras excluir NA y mantener == 1.


(316025, 38)

In [6]:
df.shape

(316025, 38)

In [7]:
print(df.columns.to_list())

['anio', 'conglome', 'vivienda', 'hogar', 'codperso', 'txt505', 'txt505b', 'txt506', 'p208a', 'p501', 'p502', 'p503', 'p504', 'p5041', 'p5042', 'p5043', 'p5044', 'p5045', 'p5046', 'p5047', 'p5048', 'p5049', 'p50410', 'p50411', 'p207', 'p301a', 'p506r4', 'p507', 'p510', 'p505r4', 'p207_label', 'p301a_label', 'p506r4_label', 'p507_label', 'p510_label', 'p505r4_label', 'p301a1', 'p301a1_label']


## Conversión de variables numéricas (coercivo y seguro)

In [8]:
# ================================================
# Convertimos variables de interes
# ================================================
# Variables de identificación (IDs) -> Las convertimos a category para ahorrar memoria
df['anio'] = df['anio'].astype('category')
df['conglome'] = df['conglome'].astype('category')
df['vivienda'] = df['vivienda'].astype('category')
df['hogar'] = df['hogar'].astype('category')
df['codperso'] = df['codperso'].astype('category')

# Variables de texto (ocupación) -> Las mantenemos como string
df.loc[df['txt505'].str.strip() == '', 'txt505'] = np.nan
df.loc[df['txt505b'].str.strip() == '', 'txt505b'] = np.nan
df.loc[df['txt506'].str.strip() == '', 'txt506'] = np.nan
df['txt505'] = df['txt505'].astype('string')
df['txt505b'] = df['txt505b'].astype('string')
df['txt506'] = df['txt506'].astype('string')

# Convertimos a string y rellenamos con ceros a la izquierda. Luego a category
df['p505r4'] = df['p505r4'].fillna(0).astype('int64').astype('str').str.zfill(4)
df['p506r4'] = df['p506r4'].fillna(0).astype('int64').astype('str').str.zfill(4)

df['p505r4'] = df['p505r4'].astype('category')
df['p506r4'] = df['p506r4'].astype('category')

# Convertimos a category las de pocas categorias
df['p301a'] = df['p301a'].astype('category')   # 12 categorías
df['p301a1'] = df['p301a1'].astype('category')
df['p507'] = df['p507'].astype('category')      # 7 categorías
df['p510'] = df['p510'].astype('category')      # 6 categorías

# Convertimos a string los labels
df['p207_label'] = df['p207_label'].astype('string')
df['p301a_label'] = df['p301a_label'].astype('string')
df['p505r4_label'] = df['p505r4_label'].astype('string')
df['p506r4_label'] = df['p506r4_label'].astype('string')
df['p507_label'] = df['p507_label'].astype('string')
df['p510_label'] = df['p510_label'].astype('string')
df['p301a1_label'] = df['p301a1_label'].astype('string')

print("Tipos tras conversión:")
print(df.dtypes)

Tipos tras conversión:
anio                  category
conglome              category
vivienda              category
hogar                 category
codperso              category
txt505          string[python]
txt505b         string[python]
txt506          string[python]
p208a                     int8
p501                   float64
p502                   float64
p503                   float64
p504                   float64
p5041                  float64
p5042                  float64
p5043                  float64
p5044                  float64
p5045                  float64
p5046                  float64
p5047                  float64
p5048                  float64
p5049                  float64
p50410                 float64
p50411                 float64
p207                      int8
p301a                 category
p506r4                category
p507                  category
p510                  category
p505r4                category
p207_label      string[python]
p301a_label     

In [9]:
print(df.columns.to_list())

['anio', 'conglome', 'vivienda', 'hogar', 'codperso', 'txt505', 'txt505b', 'txt506', 'p208a', 'p501', 'p502', 'p503', 'p504', 'p5041', 'p5042', 'p5043', 'p5044', 'p5045', 'p5046', 'p5047', 'p5048', 'p5049', 'p50410', 'p50411', 'p207', 'p301a', 'p506r4', 'p507', 'p510', 'p505r4', 'p207_label', 'p301a_label', 'p506r4_label', 'p507_label', 'p510_label', 'p505r4_label', 'p301a1', 'p301a1_label']


## Separando variables de interés

In [10]:
# Separando las variables de interes
cols_of_interest = ['anio', 'conglome', 'vivienda', 'hogar', 'codperso', 'p207', 'p207_label', 'p208a', 'p301a', 'p301a_label', 'p301a1', 'p301a1_label', 'p505r4', 'p505r4_label', 'txt505', 'txt505b', 'p506r4', 'p506r4_label', 'txt506', 'p507', 'p507_label', 'p510', 'p510_label']
df = df[cols_of_interest]

In [11]:
# ===========================================================
# Verificación de columnas seleccionadas y orden final
# ===========================================================
print(f"📊 El DataFrame actual tiene {df.shape[1]} columnas:")

for i, col in enumerate(df.columns, start=1):
    print(f"{i:>2}. {col}")

📊 El DataFrame actual tiene 23 columnas:
 1. anio
 2. conglome
 3. vivienda
 4. hogar
 5. codperso
 6. p207
 7. p207_label
 8. p208a
 9. p301a
10. p301a_label
11. p301a1
12. p301a1_label
13. p505r4
14. p505r4_label
15. txt505
16. txt505b
17. p506r4
18. p506r4_label
19. txt506
20. p507
21. p507_label
22. p510
23. p510_label


In [12]:
df.shape

(316025, 23)

In [13]:
print(df.columns.to_list(),"\n")
print(df.dtypes)

['anio', 'conglome', 'vivienda', 'hogar', 'codperso', 'p207', 'p207_label', 'p208a', 'p301a', 'p301a_label', 'p301a1', 'p301a1_label', 'p505r4', 'p505r4_label', 'txt505', 'txt505b', 'p506r4', 'p506r4_label', 'txt506', 'p507', 'p507_label', 'p510', 'p510_label'] 

anio                  category
conglome              category
vivienda              category
hogar                 category
codperso              category
p207                      int8
p207_label      string[python]
p208a                     int8
p301a                 category
p301a_label     string[python]
p301a1                category
p301a1_label    string[python]
p505r4                category
p505r4_label    string[python]
txt505          string[python]
txt505b         string[python]
p506r4                category
p506r4_label    string[python]
txt506          string[python]
p507                  category
p507_label      string[python]
p510                  category
p510_label      string[python]
dtype: object


In [14]:
df.tail(5)

,anio,conglome,vivienda,hogar,codperso,p207,p207_label,p208a,p301a,p301a_label,...,p505r4_label,txt505,txt505b,p506r4,p506r4_label,txt506,p507,p507_label,p510,p510_label
438422,2024,021001,026,11,03,1,hombre,17,6.0,secundaria completa,...,vendedores minoristas en tiendas y establecimi...,AYUDANTE DE VENDEDOR DE TELA,ATENDER CLIENTE,4751,venta al por menor de productos textiles en al...,VENTA DE TELA AL POR MENOR ENTIENDA,3.0,empleado,6.0,empresa o patrono privado
438423,2024,021001,026,11,04,2,mujer,15,5.0,secundaria incompleta,...,peones de explotaciones agrÃ­colas y ganaderas,AYUDANTE DE ÑRODUCTOR,CISECHAR PAP,0113,"cultivo de vegetales y melones, raÃ­ces y tubÃ...",CULTIVO DE PAPA,5.0,trabajador familiar no remunerado,NaN,<NA>
438424,2024,021001,029,11,01,1,hombre,80,3.0,primaria incompleta,...,agricultores y trabajadores calificados de cul...,PRODUCTOR,AGROPECUARIO,0113,"cultivo de vegetales y melones, raÃ­ces y tubÃ...",CULTIVO DE PAPA,2.0,trabajador independiente,NaN,<NA>
438425,2024,021001,037,11,01,1,hombre,56,5.0,secundaria incompleta,...,agricultores y trabajadores calificados de cul...,PRODUCTOR AGROPECUARIO,APORTAR PAPA,0150,cultivo de productos agrÃ­colas en combinaciÃ³...,CULTIVA PAPA HABA AVENA FORRAJE IZAÑO CRIANZA ...,2.0,trabajador independiente,NaN,<NA>
438426,2024,021001,037,11,02,2,mujer,54,3.0,primaria incompleta,...,peones de explotaciones agrÃ­colas y ganaderas,AYUDANTE DE PRODUCTOR AGROPECUARIO,ALIMENTAR PORCINO AMARRAR VACUNO OVINO ORDEÑAR...,0150,cultivo de productos agrÃ­colas en combinaciÃ³...,CULTIVP DE PAPA AVENA FORRAJERA ALFA CRANZA DE...,5.0,trabajador familiar no remunerado,NaN,<NA>


In [15]:
print("Tamaño real:", df.shape)
print("Primeros índices:", df.index[:5].tolist())
print("Últimos índices:", df.index[-5:].tolist())
print("Índice único y ordenado:", df.index.is_monotonic_increasing and df.index.is_unique)

Tamaño real: (316025, 23)
Primeros índices: [0, 2, 3, 6, 7]
Últimos índices: [438422, 438423, 438424, 438425, 438426]
Índice único y ordenado: True


In [16]:
df = df.reset_index(drop=True)

In [17]:
df.tail()

,anio,conglome,vivienda,hogar,codperso,p207,p207_label,p208a,p301a,p301a_label,...,p505r4_label,txt505,txt505b,p506r4,p506r4_label,txt506,p507,p507_label,p510,p510_label
316020,2024,021001,026,11,03,1,hombre,17,6.0,secundaria completa,...,vendedores minoristas en tiendas y establecimi...,AYUDANTE DE VENDEDOR DE TELA,ATENDER CLIENTE,4751,venta al por menor de productos textiles en al...,VENTA DE TELA AL POR MENOR ENTIENDA,3.0,empleado,6.0,empresa o patrono privado
316021,2024,021001,026,11,04,2,mujer,15,5.0,secundaria incompleta,...,peones de explotaciones agrÃ­colas y ganaderas,AYUDANTE DE ÑRODUCTOR,CISECHAR PAP,0113,"cultivo de vegetales y melones, raÃ­ces y tubÃ...",CULTIVO DE PAPA,5.0,trabajador familiar no remunerado,NaN,<NA>
316022,2024,021001,029,11,01,1,hombre,80,3.0,primaria incompleta,...,agricultores y trabajadores calificados de cul...,PRODUCTOR,AGROPECUARIO,0113,"cultivo de vegetales y melones, raÃ­ces y tubÃ...",CULTIVO DE PAPA,2.0,trabajador independiente,NaN,<NA>
316023,2024,021001,037,11,01,1,hombre,56,5.0,secundaria incompleta,...,agricultores y trabajadores calificados de cul...,PRODUCTOR AGROPECUARIO,APORTAR PAPA,0150,cultivo de productos agrÃ­colas en combinaciÃ³...,CULTIVA PAPA HABA AVENA FORRAJE IZAÑO CRIANZA ...,2.0,trabajador independiente,NaN,<NA>
316024,2024,021001,037,11,02,2,mujer,54,3.0,primaria incompleta,...,peones de explotaciones agrÃ­colas y ganaderas,AYUDANTE DE PRODUCTOR AGROPECUARIO,ALIMENTAR PORCINO AMARRAR VACUNO OVINO ORDEÑAR...,0150,cultivo de productos agrÃ­colas en combinaciÃ³...,CULTIVP DE PAPA AVENA FORRAJERA ALFA CRANZA DE...,5.0,trabajador familiar no remunerado,NaN,<NA>


## Validaciones rápidas de integridad

In [18]:
# ================================================
# Validaciones de integridad y rangos
# ================================================
# --- Resumen tabular de columnas str ---
def summarize_string(df):
    rows = []
    str_cols = df.select_dtypes(include=["object", "string"]).columns

    for c in str_cols:
        s = df[c].astype("string")

        lens = s.dropna().str.len()

        row = {
            "col": c,
            "dtype": df[c].dtype.name,
            "n": len(s),
            "n_missing": int(s.isna().sum()),
            "%missing": 100 * s.isna().mean(),
            "n_unique": s.nunique(dropna=True),
            "len_min": lens.min(skipna=True),
            "len_p25": lens.quantile(0.25),
            "len_p50": lens.quantile(0.50),
            "len_p75": lens.quantile(0.75),
            "len_max": lens.max(skipna=True),
        }
        rows.append(row)

    out = pd.DataFrame(rows)
    out = out.sort_values("%missing", ascending=False).reset_index(drop=True)
    return out


str_summary = summarize_string(df)

str_summary.style.format({
    "%missing": "{:.2f}",
    "len_min": "{:.0f}", "len_p25": "{:.0f}",
    "len_p50": "{:.0f}", "len_p75": "{:.0f}", "len_max": "{:.0f}"
})

,col,dtype,n,n_missing,%missing,n_unique,len_min,len_p25,len_p50,len_p75,len_max
0,p301a1_label,string,316025,233788,73.98,745,5,12,18,25,120
1,p510_label,string,316025,197070,62.36,6,4,25,25,25,56
2,txt505b,string,316025,427,0.14,199876,1,23,33,48,70
3,p301a_label,string,316025,79,0.02,12,7,19,19,31,36
4,p207_label,string,316025,0,0.00,2,5,5,6,6,6
5,p505r4_label,string,316025,0,0.00,450,5,34,46,69,119
6,txt505,string,316025,0,0.00,65106,1,17,21,30,70
7,p506r4_label,string,316025,0,0.00,392,6,43,59,93,124
8,txt506,string,316025,0,0.00,165615,3,35,49,63,70
9,p507_label,string,316025,0,0.00,7,4,8,24,24,33


In [19]:
# --- Resumen tabular de columnas numéricas ---

def summarize_numeric(df):
    rows = []
    num_cols = df.select_dtypes(include=["number"]).columns

    for c in num_cols:
        s = df[c]

        # Convertir categorías a numérico (códigos)
        if isinstance(s.dtype, pd.CategoricalDtype):
            s = s.cat.codes.replace(-1, np.nan)

        s = pd.to_numeric(s, errors="coerce")

        row = {
            "col": c,
            "dtype": df[c].dtype.name,
            "n": len(s),
            "n_missing": int(s.isna().sum()),
            "%missing": 100 * s.isna().mean(),
            "n_unique": s.nunique(dropna=True),
            "min": s.min(skipna=True),
            "p01": s.quantile(0.01),
            "p25": s.quantile(0.25),
            "p50": s.quantile(0.50),
            "p75": s.quantile(0.75),
            "p99": s.quantile(0.99),
            "max": s.max(skipna=True),
            "mean": s.mean(skipna=True),
            "std": s.std(skipna=True),
        }
        rows.append(row)

    out = pd.DataFrame(rows)
    out = out.sort_values("%missing", ascending=False).reset_index(drop=True)
    return out


num_summary = summarize_numeric(df)

num_summary.style.format({
    "%missing": "{:.2f}",
    "min": "{:.0f}", "p01": "{:.0f}", "p25": "{:.0f}", "p50": "{:.0f}",
    "p75": "{:.0f}", "p99": "{:.0f}", "max": "{:.0f}",
    "mean": "{:.2f}", "std": "{:.2f}"
})

,col,dtype,n,n_missing,%missing,n_unique,min,p01,p25,p50,p75,p99,max,mean,std
0,p207,int8,316025,0,0.00,2,1,1,1,1,2,2,2,1.48,0.50
1,p208a,int8,316025,0,0.00,85,14,14,29,42,55,82,98,42.47,17.09


In [20]:
# --- Resumen tabular de columnas category ---

def summarize_category(df, top_k=5):
    """
    Resume columnas de tipo 'category'.
    Para cada columna devuelve:
      - n, n_missing, %missing, n_unique
      - is_ordered
      - n_codes (número de categorías definidas)
      - top_k_labels (string "label1 (pct%), label2 (pct%), ...")
      - top_k_counts (string "count1, count2, ...")
    """
    rows = []
    cat_cols = df.select_dtypes(include=["category"]).columns

    for c in cat_cols:
        s = df[c].astype("category")
        n = len(s)
        n_missing = int(s.isna().sum())
        pct_missing = 100 * s.isna().mean()
        n_unique = s.nunique(dropna=True)
        is_ordered = bool(s.cat.ordered)
        n_codes = len(s.cat.categories)

        # top_k con porcentajes sobre el total no-missing (o sobre n si prefieres)
        vc = s.dropna().value_counts()
        top = vc.head(top_k)
        top_labels = []
        top_counts = []
        for label, cnt in top.items():
            pct = 100 * cnt / (n - n_missing) if (n - n_missing) > 0 else 0.0
            top_labels.append(f"{label} ({pct:.1f}%)")
            top_counts.append(str(int(cnt)))

        row = {
            "col": c,
            "dtype": df[c].dtype.name,
            "n": n,
            "n_missing": n_missing,
            "%missing": pct_missing,
            "n_unique": n_unique,
            "is_ordered": is_ordered,
            "n_codes": n_codes,
            f"top_{top_k}_labels": ", ".join(top_labels) if top_labels else "",
            f"top_{top_k}_counts": ", ".join(top_counts) if top_counts else ""
        }
        rows.append(row)

    out = pd.DataFrame(rows)
    out = out.sort_values("%missing", ascending=False).reset_index(drop=True)
    return out

# Uso:
cat_summary = summarize_category(df, top_k=5)

# Mostrar con formato
cat_summary.style.format({
    "%missing": "{:.2f}"
})

,col,dtype,n,n_missing,%missing,n_unique,is_ordered,n_codes,top_5_labels,top_5_counts
0,p301a1,category,316025,233788,73.98,802,False,802,"714035.0 (6.4%), 342056.0 (3.9%), 526165.0 (3.4%), 342025.0 (3.4%), 351016.0 (3.4%)","5255, 3176, 2826, 2821, 2789"
1,p510,category,316025,197070,62.36,6,False,6,"6.0 (75.8%), 2.0 (20.9%), 5.0 (1.6%), 1.0 (1.1%), 3.0 (0.5%)","90200, 24906, 1929, 1317, 602"
2,p301a,category,316025,79,0.02,12,False,12,"6.0 (25.8%), 5.0 (15.6%), 3.0 (13.8%), 4.0 (12.1%), 8.0 (8.6%)","81622, 49311, 43496, 38234, 27038"
3,vivienda,category,316025,0,0.00,766,False,766,"010 (0.9%), 003 (0.9%), 002 (0.9%), 009 (0.9%), 004 (0.9%)","2986, 2904, 2865, 2846, 2805"
4,conglome,category,316025,0,0.00,9189,False,9189,"016630 (0.0%), 015094 (0.0%), 017755 (0.0%), 017776 (0.0%), 018758 (0.0%)","133, 131, 125, 121, 121"
5,anio,category,316025,0,0.00,5,False,5,"2022 (20.6%), 2021 (20.2%), 2023 (20.1%), 2024 (19.9%), 2020 (19.2%)","65240, 63681, 63587, 62874, 60643"
6,codperso,category,316025,0,0.00,18,False,18,"01 (44.8%), 02 (30.5%), 03 (14.0%), 04 (6.1%), 05 (2.4%)","141672, 96413, 44354, 19299, 7661"
7,hogar,category,316025,0,0.00,15,False,15,"11 (98.3%), 12 (0.8%), 22 (0.6%), 13 (0.1%), 33 (0.1%)","310611, 2444, 1851, 377, 272"
8,p505r4,category,316025,0,0.00,450,False,450,"9211 (20.8%), 6114 (13.8%), 5212 (7.6%), 5120 (2.9%), 9313 (2.6%)","65703, 43753, 23921, 9101, 8282"
9,p506r4,category,316025,0,0.00,392,False,392,"0150 (27.9%), 5610 (5.7%), 4721 (4.8%), 4100 (3.6%), 4922 (3.2%)","88074, 17939, 15076, 11499, 10131"


In [21]:
# ===========================================================
# ✅ Verificación de reglas de negocio (versión corregida y robusta)
# ===========================================================
checks = []

# --- Reglas numéricas ---
if "anio" in df:
    s = pd.to_numeric(df["anio"], errors="coerce")  # 🔹 Conversión segura a numérico
    ok = s.between(2020, 2024).all()
    checks.append({
        "check": "anio in [2020,2024]",
        "status": "OK" if ok else "ALERTA",
        "detalle": f"min={s.min()}, max={s.max()}, n_missing={s.isna().sum()}"
    })

if "p208a" in df:
    s = pd.to_numeric(df["p208a"], errors="coerce")
    ok = s.dropna().between(10, 99).all()
    checks.append({
        "check": "p208a in [10,99]",
        "status": "OK" if ok else "ALERTA",
        "detalle": f"min={int(s.min())}, p01={int(s.quantile(0.01))}, p99={int(s.quantile(0.99))}, max={int(s.max())}, n_missing={s.isna().sum()}"
    })

for c in ["p505r4", "p506r4"]:
    if c in df:
        s = pd.to_numeric(df[c], errors="coerce")
        ok = (s.dropna() > 0).all()
        checks.append({
            "check": f"{c} > 0",
            "status": "OK" if ok else "ALERTA",
            "detalle": f"min={int(s.min())}, max={int(s.max())}, n_missing={int(s.isna().sum())}"
        })

if "p207" in df:
    s = pd.to_numeric(df["p207"], errors="coerce")
    vals = sorted(s.dropna().unique().tolist())
    ok = set(vals).issubset({1, 2})
    checks.append({
        "check": "p207 in {1,2}",
        "status": "OK" if ok else "ALERTA",
        "detalle": f"valores={vals}, n_missing={s.isna().sum()}"
    })

# --- Reglas categóricas ---
for col in df.select_dtypes("category"):
    n_unique = df[col].nunique(dropna=True)
    top_val = df[col].value_counts(dropna=True).index[0] if n_unique > 0 else None
    top_freq = df[col].value_counts(dropna=True).iloc[0] if n_unique > 0 else 0
    checks.append({
        "check": f"{col} categoría",
        "status": "OK" if n_unique > 0 else "ALERTA",
        "detalle": f"n_unique={n_unique}, top='{top_val}', top_freq={top_freq}, n_missing={df[col].isna().sum()}"
    })

# --- Reglas de texto ---
for col in df.select_dtypes("object"):
    s = df[col].astype(str)
    lens = s.str.len()
    checks.append({
        "check": f"{col} longitud texto",
        "status": "OK" if lens.dropna().mean() > 0 else "ALERTA",
        "detalle": f"len_min={lens.min()}, len_p25={lens.quantile(0.25):.1f}, len_p50={lens.median():.1f}, len_p75={lens.quantile(0.75):.1f}, len_max={lens.max()}, n_missing={df[col].isna().sum()}"
    })

# --- Resultado final ---
checks_df = pd.DataFrame(checks)
display(checks_df)


,check,status,detalle
0,"anio in [2020,2024]",OK,"min=2020, max=2024, n_missing=0"
1,"p208a in [10,99]",OK,"min=14, p01=14, p99=82, max=98, n_missing=0"
2,p505r4 > 0,OK,"min=111, max=9629, n_missing=0"
3,p506r4 > 0,OK,"min=111, max=9900, n_missing=0"
4,"p207 in {1,2}",OK,"valores=[1, 2], n_missing=0"
5,anio categoría,OK,"n_unique=5, top='2022', top_freq=65240, n_miss..."
6,conglome categoría,OK,"n_unique=9189, top='016630', top_freq=133, n_m..."
7,vivienda categoría,OK,"n_unique=766, top='010', top_freq=2986, n_miss..."
8,hogar categoría,OK,"n_unique=15, top='11', top_freq=310611, n_miss..."
9,codperso categoría,OK,"n_unique=18, top='01', top_freq=141672, n_miss..."


## Tratamiento de missings

In [22]:
((df.isna().sum()*100) / df.shape[0]).loc[lambda x : x > 0].sort_values(ascending=False).round(2)

p301a1          73.98
p301a1_label    73.98
p510            62.36
p510_label      62.36
txt505b          0.14
p301a_label      0.02
p301a            0.02
dtype: float64

### Variables `301a_label` y `301a1_label`

- `301a_label`: Último año o grado de estudios y nivel que aprobó -> 0.02% de missings

- `301a1_label`: Carrera superior universitaria o no universitaria que ud. estudia o ha estudiado -> 73.98% de missings

In [23]:
# Observar presencia exclusiva de NAs

df[df['p301a1_label'].isna()][['p301a_label', 'p301a1_label']].drop_duplicates()

,p301a_label,p301a1_label
0,primaria completa,<NA>
2,secundaria completa,<NA>
7,superior no universitaria incompleta,<NA>
9,superior no universitaria completa,<NA>
14,primaria incompleta,<NA>
22,sin nivel,<NA>
38,superior universitaria completa,<NA>
49,secundaria incompleta,<NA>
61,superior universitaria incompleta,<NA>
73,maestria/doctorado,<NA>


In [24]:
# Lista de valores que quieres filtrar
valores_filtrados = [
    'superior no universitaria incompleta',
    'superior no universitaria completa',
    'superior universitaria incompleta',
    'superior universitaria completa'
]

# Filtrar el DataFrame con esas categorías
df_filtrado = df[df['p301a_label'].isin(valores_filtrados)]

# Calcular % de missings por cada p301a_label
porcentaje_missings = (
    df_filtrado['p301a1_label'].isna()
    .groupby(df_filtrado['p301a_label'])
    .mean()
    .reset_index(name='porc_missing_p301a1')
)

# Convertir a porcentaje
porcentaje_missings['porc_missing_p301a1'] = (
    porcentaje_missings['porc_missing_p301a1'] * 100
).round(2)

# Mostrar resultado
print(porcentaje_missings)


                            p301a_label  porc_missing_p301a1
0    superior no universitaria completa                10.57
1  superior no universitaria incompleta                10.30
2       superior universitaria completa                 9.37
3     superior universitaria incompleta                 8.42


Es decir, en realidad tenemos un 38.66% de missings para la variable `301a1_label` (Carrera superior universitaria o no universitaria que ud. estudia o ha estudiado).

### Variable `p510_label`

- `p510_label`: Tipo de entidad para la que trabajó en su ocupación principal.

El encuestador llena esta pregunta si en la pregunta `p507_label` (Ud. se desempeñó en su ocupación principal como) marcó "Empleado", "Obrero" u "Otro".

In [25]:
# Observar presencia exclusiva de NAs

df[df['p510_label'].isna()][['p507_label', 'p510_label']].drop_duplicates()

,p507_label,p510_label
0,trabajador independiente,<NA>
2,empleador o patrono,<NA>
21,trabajador familiar no remunerado,<NA>
23,trabajador del hogar,<NA>


In [26]:
df['p510_label'].isna().groupby(df['p507_label']).value_counts()


p507_label                         p510_label
empleado                           False          57552
empleador o patrono                True           10072
obrero                             False          59889
otro                               False           1514
trabajador del hogar               True            4768
trabajador familiar no remunerado  True           59152
trabajador independiente           True          123078
Name: count, dtype: int64

In [27]:
tabla = (
    df.assign(missing=df['p510_label'].isna())
    .groupby('p507_label')['missing']
    .agg(total='count', n_missing='sum')
    .reset_index()
)

tabla['porc_missing_p510'] = (tabla['n_missing'] / tabla['total'] * 100).round(2)

print(tabla)


                          p507_label   total  n_missing  porc_missing_p510
0                           empleado   57552          0                0.0
1                empleador o patrono   10072      10072              100.0
2                             obrero   59889          0                0.0
3                               otro    1514          0                0.0
4               trabajador del hogar    4768       4768              100.0
5  trabajador familiar no remunerado   59152      59152              100.0
6           trabajador independiente  123078     123078              100.0


Validamos que sí tenemos todos los valores (100%) para la `p510_label` cuando en la pregunta `p507_label` marcaron "Empleado", "Obrero" u "Otro".

### Variable `txt505b`

- `txt505b`: Tareas que realizó en su ocupación principal.

In [28]:
df[df['txt505b'].isna()][['txt505b', 'txt505']].drop_duplicates()

,txt505b,txt505
85,<NA>,PEON DE ALBAÑIL
1642,<NA>,CONDUCTOR DE VEHICULO
1656,<NA>,COCINERO
1744,<NA>,RECEPCIONISTA
1864,<NA>,PEON DE CULTIVO
...,...,...
60340,<NA>,PROFESOR DE PRIMARIA
108787,<NA>,SUB OFICIAL TÉCNICO DE PRIMERA
113594,<NA>,PRODUCTOR AGROPECUARIO
223959,<NA>,TRABAJADORA DEL HOGAR


Tenemos un 0.14% de missings en la `txt505b`, lo cual puede reducir el contexto para las distintas 98 ocupaciones.

## Limpieza de data



In [29]:
# Distribución de longitudes de literales en base a caracteres.
longitudes = df['txt505'].astype(str).str.len()
longitudes.describe()

count    316025.000000
mean         23.784786
std          11.830971
min           1.000000
25%          17.000000
50%          21.000000
75%          30.000000
max          70.000000
Name: txt505, dtype: float64

In [30]:
# Literales que contienen numeros
print(df['txt505'][df['txt505'].str.contains(r'\d+', na=False)].shape[0])
df['txt505'][df['txt505'].str.contains(r'\d+', na=False)]

12


30823                                     PE0N AGROPECUARIO
47403                                 CONDUCT0R DE TRICICLO
70778                   ELABORADOR DE MAQUETA VIRTUAL EN 3D
104483                                   SUB OFICIAL DE 3RA
119099                                                    5
120887                                                    4
125011    ASESOR DE ESTUDIENTES PARA MANEJO DE PROGRAMAS...
161763                                  AUXILIAR DE 0FICINA
183231                                   SUB OFICIAL DE 2DA
208595                                        ASES0RA LEGAL
231030                                CHOFER  DE CAMIO0NETA
311454                VENDEDORA DE CERVEZA GASEOSA AGUA MI5
Name: txt505, dtype: string

In [31]:
# Literales que presentan solo numeros
df[~df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False)]

,anio,conglome,vivienda,hogar,codperso,p207,p207_label,p208a,p301a,p301a_label,...,p505r4_label,txt505,txt505b,p506r4,p506r4_label,txt506,p507,p507_label,p510,p510_label
119099,2021,019731,024,11,03,2,mujer,15,5.0,secundaria incompleta,...,"panaderos, pasteleros y confiteros",5,PREPARAR DECORAR TORTA,1071,elaboraciÃ³n de productos de panaderÃ­a,PREPARACION VENTA DE TORTAS EN CASA DEL CLIENTES,2.0,trabajador independiente,NaN,<NA>
120887,2021,019955,039,11,02,2,mujer,38,1.0,sin nivel,...,vendedores en kiosco fijo y puestos de mercado,4,VENDER OFRECER COBRAR ATENDER,4782,"venta al por menor de productos textiles, pren...",VENTA DE ZAPATILLA ZAPATO AL POR MENOR EN PUES...,3.0,empleado,6.0,empresa o patrono privado


In [32]:
# Eliminacion de Literales que presentan solo numeros
df = df[df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False)].copy()

In [33]:
# Literales con 2 caracteres o menos
df[
    df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False) &
    (df['txt505'].str.len() <= 2)
][['txt505', 'p505r4','txt505b','p506r4_label']]

,txt505,p505r4,txt505b,p506r4_label
16748,DJ,3521,ARMAR EQUIPO PONER MEZCLAR MÚSICA,"actividades de arte, entretenimiento y creativ..."
38119,DJ,3521,HACER MEZCLA DE MUSICA ANIMAR,"actividades de arte, entretenimiento y creativ..."
153615,DJ,3521,PROGRAMAR MÚSICA EN UNA EN UNA FIESTA SOCIAL C...,"actividades de arte, entretenimiento y creativ..."
176176,DJ,3521,CLASIFICAR DISCO PONER MUSICA EN DISCOTECA,otras actividades de diversiÃ³n y esparcimient...
183372,DJ,3521,MESCLAR MUSICA PONER MUSICA,otras actividades de diversiÃ³n y esparcimient...
195123,DJ,3521,AYUDAR ACOMODAR EQUIPOS MEZCLAR MUSICA,otras actividades de diversiÃ³n y esparcimient...
216573,DJ,3521,PROGRAMAR LA MUSICA DE LA DISCOTECA,otras actividades de diversiÃ³n y esparcimient...
223569,DJ,3521,REALIZAR MEZCLA DE MUSICA,"actividades de arte, entretenimiento y creativ..."
231940,DJ,3521,PONER MUSIC,otras actividades de diversiÃ³n y esparcimient...
233531,DJ,3521,PONER MUSICA EQUALIZAR SONIDA,"actividades de arte, entretenimiento y creativ..."


In [34]:
# Literales con 3 caracteres o menos que no incluya los anteriores revisados
df[
    df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False) &
    (df['txt505'].str.len() <= 3) &
    (~df['txt505'].isin(['DJ', 'DG']))
][['txt505', 'txt505b', 'p506r4_label']]

,txt505,txt505b,p506r4_label
244015,SUP,PAGAR PERSONAL PROVEEDORES ADMINISTRAR NEGOCIO,actividades de arquitectura e ingenierÃ­a y ac...


In [35]:
#Eliminacion de registro
df = df[df['txt505'].str.upper() != 'SUP'].copy()

In [36]:
# Literales con 3 caracteres o menos que no incluya los anteriores revisados
df[
    df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False) &
    (df['txt505'].str.len() <= 4) &
    (~df['txt505'].isin(['DJ', 'DG']))
][['txt505', 'txt505b', 'p506r4_label']]

,txt505,txt505b,p506r4_label
4307,CHEF,PREPARAR INGREDIENTES PELAR PICAR COCINAR,restaurantes y servicios mÃ³viles de alimentac...
4487,MOZA,ATENDER MESAS,otras actividades de diversiÃ³n y esparcimient...
4710,MOZO,ATENDER OFRECER CLIENTE LLEVAR PLATOS,restaurantes y servicios mÃ³viles de alimentac...
4969,MOZO,ATENDER CLIENTES TOMAR PEDIDOS,restaurantes y servicios mÃ³viles de alimentac...
4989,PEON,LIMPIAR CANALES SACAR BASURA,actividades de apoyo a los cultivos
...,...,...,...
313388,MOZO,LLEVAR PLATOS PASAR COPAS LAVAR PLATOS PREPARA...,provisiÃ³n de comidas preparadas
314748,PEON,LIMPIAR CANAL COSECHAR,"cultivo de vegetales y melones, raÃ­ces y tubÃ..."
314756,PEON,AYUDANTE DE CHANCADORA DE PIEDRA,construcciÃ³n de caminos y vÃ­as fÃ©rreas
315404,PEON,ECHANDO FERTILIZANTES,"cultivo de vegetales y melones, raÃ­ces y tubÃ..."


In [37]:
# Literales con 4 caracteres o menos que no incluya los anteriores revisados
df[
    df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False) &
    (df['txt505'].str.len() <= 4) &
    (~df['txt505'].isin(['DJ', 'DG', 'SUP']))
][['txt505', 'txt505b','p506r4_label']].drop_duplicates(subset='txt505').reset_index(drop=True)

,txt505,txt505b,p506r4_label
0,CHEF,PREPARAR INGREDIENTES PELAR PICAR COCINAR,restaurantes y servicios mÃ³viles de alimentac...
1,MOZA,ATENDER MESAS,otras actividades de diversiÃ³n y esparcimient...
2,MOZO,ATENDER OFRECER CLIENTE LLEVAR PLATOS,restaurantes y servicios mÃ³viles de alimentac...
3,PEON,LIMPIAR CANALES SACAR BASURA,actividades de apoyo a los cultivos
4,TAXI,CONDUCIR,otros tipos de transporte terrestre de pasajeros
5,MOSA,ATENDER CLIENTE PREPARAR REFRESCO LIMPIAR LOCAL,restaurantes y servicios mÃ³viles de alimentac...
6,JUEZ,LLEVAR ACABO JUCIOS AUDIENCIAS,actividades de mantenimiento del orden pÃºblic...
7,BUZO,EXTRAER PESCADO DEL MAR,pesca marÃ­tima
8,JEFE,GESTIONAR TRAMITAR DIRIGIR,actividades de la administraciÃ³n pÃºblica en ...
9,MOSO,MOSEAR TOMAR ENTREGAR PEDIDOS,restaurantes y servicios mÃ³viles de alimentac...


In [38]:
#CORREGIR CASOS OBVIOS
correcciones = {
    'DG': 'DJ',
    'MOSO': 'MOZO',
    'MOSA': 'MOZA',
    'MOZQ': 'MOZA',   # caso detectado
    'PEON': 'PEÓN',
    'CAJA': 'CAJERA',  # aquí depende del contexto, si quieres lo conversamos
}
df['txt505'] = df['txt505'].replace(correcciones)

In [39]:
# Literales con 4 caracteres o menos que no incluya los anteriores revisados
df[
    df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False) &
    (df['txt505'].str.len() <= 4) &
    (~df['txt505'].isin(['DJ', 'DG', 'SUP']))
][['txt505', 'txt505b','p506r4_label']].drop_duplicates(subset='txt505').reset_index(drop=True)

,txt505,txt505b,p506r4_label
0,CHEF,PREPARAR INGREDIENTES PELAR PICAR COCINAR,restaurantes y servicios mÃ³viles de alimentac...
1,MOZA,ATENDER MESAS,otras actividades de diversiÃ³n y esparcimient...
2,MOZO,ATENDER OFRECER CLIENTE LLEVAR PLATOS,restaurantes y servicios mÃ³viles de alimentac...
3,PEÓN,LIMPIAR CANALES SACAR BASURA,actividades de apoyo a los cultivos
4,TAXI,CONDUCIR,otros tipos de transporte terrestre de pasajeros
5,JUEZ,LLEVAR ACABO JUCIOS AUDIENCIAS,actividades de mantenimiento del orden pÃºblic...
6,BUZO,EXTRAER PESCADO DEL MAR,pesca marÃ­tima
7,JEFE,GESTIONAR TRAMITAR DIRIGIR,actividades de la administraciÃ³n pÃºblica en ...
8,NANA,CUIDAR DAR DE COMER,actividades de los hogares en calidad de emple...


In [40]:
# Literales con 5 caracteres o menos que no incluya los anteriores revisados
df[
    df['txt505'].str.contains(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ]', na=False) &
    (df['txt505'].str.len() <= 5) &
    (~df['txt505'].isin(['DJ', 'DG', 'SUP']))
][['txt505', 'txt505b','p506r4_label']].drop_duplicates(subset='txt505').reset_index(drop=True)

,txt505,txt505b,p506r4_label
0,CHEF,PREPARAR INGREDIENTES PELAR PICAR COCINAR,restaurantes y servicios mÃ³viles de alimentac...
1,MOZA,ATENDER MESAS,otras actividades de diversiÃ³n y esparcimient...
2,MOZO,ATENDER OFRECER CLIENTE LLEVAR PLATOS,restaurantes y servicios mÃ³viles de alimentac...
3,PEÓN,LIMPIAR CANALES SACAR BASURA,actividades de apoyo a los cultivos
4,TAXI,CONDUCIR,otros tipos de transporte terrestre de pasajeros
5,CHEFF,PREPARAR COMIDA,restaurantes y servicios mÃ³viles de alimentac...
6,VIGIA,CONTROLAR PASE DE INGRESO Y SALIDA DE LOS VEHI...,construcciÃ³n de caminos y vÃ­as fÃ©rreas
7,JUEZ,LLEVAR ACABO JUCIOS AUDIENCIAS,actividades de mantenimiento del orden pÃºblic...
8,BUZO,EXTRAER PESCADO DEL MAR,pesca marÃ­tima
9,DIJEY,PROGRAMAR MUSICA ANIMAR MUSICA EN REUNIONES PA...,"actividades de arte, entretenimiento y creativ..."


## Limpieza lingüística

**Objetivo**: corregir de manera programática, masiva y lingüísticamente robusta las tres columnas de texto (`txt505`, `txt505b`, `txt506`):

- Errores de codificación (como “comunicaciÃ³n” → “comunicación” y “niÃ±o” → “niño”).
→ Esto se debe a lecturas UTF-8 interpretadas como Latin-1 (mojibake).

- Palabras que deben llevar tilde obligatoriamente (como comunicación, educación, situación…).
→ Solo esas, no los casos donde el acento es opcional (si/sí, solo/sólo).

- Ñ incorrectas o perdidas (ej. nino → niño, espana → españa).

- Espacios múltiples, caracteres invisibles o de control.

In [41]:
pip install pandas ftfy pyspellchecker wordfreq unidecode python-Levenshtein swifter tqdm pyarrow fastparquet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
# ===========================================================
# BLOQUE FINAL – Limpieza y corrección lingüística avanzada (versión estable combinada)
# ===========================================================
!pip install ftfy pyspellchecker wordfreq unidecode python-Levenshtein swifter > /dev/null

import pandas as pd, re, unicodedata, ftfy, Levenshtein, swifter
from spellchecker import SpellChecker
from wordfreq import zipf_frequency, top_n_list
from unidecode import unidecode

# -----------------------------------------------------------
# 0️⃣ Configuración base
# -----------------------------------------------------------
df_sub = df.copy()   # ⚙️ Cambia a df para correr toda la base

spell = SpellChecker(language="es")
spanish_vocab = set(top_n_list("es", 100_000))

# -----------------------------------------------------------
# 1️⃣ Restaurar “ñ” (versión robusta)
# -----------------------------------------------------------
def restaurar_n(text):
    words, corrected = text.split(), []
    for w in words:
        if not w.isalpha():
            corrected.append(w)
            continue
        if "ñ" in w:
            corrected.append(w)
            continue

        # Probamos reemplazar letras "n" por "ñ"
        variants = {w[:i]+"ñ"+w[i+1:] for i,ch in enumerate(w) if ch=="n"}
        valid = [v for v in variants if (zipf_frequency(v, "es") > 1.8 and v in spanish_vocab)]
        if valid and zipf_frequency(valid[0], "es") > zipf_frequency(w, "es"):
            corrected.append(valid[0])
        else:
            corrected.append(w)
    return " ".join(corrected)

# -----------------------------------------------------------
# 2️⃣ Restaurar tildes (versión híbrida, evita verbos conjugados)
# -----------------------------------------------------------
def restaurar_tildes(text):
    words, corrected = text.split(), []
    for w in words:
        if not w.isalpha():
            corrected.append(w)
            continue

        plain = unidecode(w)
        if plain == w:
            best_variant, best_freq = None, 0
            for base, acc in zip("aeiou", "áéíóú"):
                if base in w:
                    variant = w.replace(base, acc)
                    f = zipf_frequency(variant, "es")
                    if f > best_freq:
                        best_variant, best_freq = variant, f

            if best_variant:
                termina = w[-3:]
                termina_vocal = w[-1] in "aeiou"
                es_verbo = termina in ["o", "as", "os", "amos", "emos", "imos"]

                # --- Evita acentuar verbos o sustantivos terminados en vocal
                if not es_verbo and (not termina_vocal) and best_freq - zipf_frequency(w, "es") > 0.8:
                    corrected.append(best_variant)
                else:
                    corrected.append(w)
            else:
                corrected.append(w)
        else:
            corrected.append(w)
    return " ".join(corrected)

# -----------------------------------------------------------
# 3️⃣ Corrección ortográfica segura
# -----------------------------------------------------------
def corregir_seguro(token):
    if not token.isalpha():
        return token
    if len(token) <= 3:
        return token
    if token in spanish_vocab:
        return token

    suggestion = spell.correction(token)
    if suggestion is None:
        return token

    dist = Levenshtein.distance(token, suggestion)
    freq_sug = zipf_frequency(suggestion, "es")
    freq_tok = zipf_frequency(token, "es")

    plain = unidecode(token)
    for base, acc in zip("aeiou", "áéíóú"):
        if base in token:
            variant = token.replace(base, acc)
            if zipf_frequency(variant, "es") > zipf_frequency(token, "es"):
                return variant

    if dist <= 1 and freq_sug > freq_tok + 0.5:
        return suggestion
    return token

# -----------------------------------------------------------
# 4️⃣ Limpieza y corrección integral
# -----------------------------------------------------------
def limpiar_y_corregir(text):
    if not isinstance(text, str):
        return text
    text = ftfy.fix_text(text)
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r"\s+", " ", text.lower()).strip()

    tokens = [corregir_seguro(t) for t in text.split()]
    text = " ".join(tokens)
    text = restaurar_n(text)
    text = restaurar_tildes(text)
    text = re.sub(r"\s{2,}", " ", text).strip()
    return text

# -----------------------------------------------------------
# 5️⃣ Aplicación paralelizada
# -----------------------------------------------------------
for c in ["txt505", "txt505b", "txt506"]:
    if c in df_sub.columns:
        print(f"⚙️ Corrigiendo columna: {c} ...")
        df_sub[f"{c}_clean"] = df_sub[c].fillna("").astype(str).swifter.apply(limpiar_y_corregir)

# -----------------------------------------------------------
# 6️⃣ Unificación final
# -----------------------------------------------------------
df_sub["texto_final"] = (
    df_sub["txt505_clean"].fillna("") + " " +
    df_sub["txt505b_clean"].fillna("") + " " +
    df_sub["txt506_clean"].fillna("")
).str.replace(r"\s{2,}", " ", regex=True).str.strip().replace("", pd.NA)

# -----------------------------------------------------------
# 7️⃣ Vista de resultados
# -----------------------------------------------------------
pd.set_option("display.max_colwidth", 120)
print("\n✅ Limpieza y corrección lingüística avanzada completada (versión híbrida).")
display(df_sub[["txt505","txt505b","txt506","texto_final"]].sample(10, random_state=42))


El sistema no puede encontrar la ruta especificada.
c:\Users\Hugo Jazyel\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⚙️ Corrigiendo columna: txt505 ...


Pandas Apply: 100%|██████████| 316022/316022 [1:34:17<00:00, 55.86it/s]  


⚙️ Corrigiendo columna: txt505b ...


Pandas Apply: 100%|██████████| 316022/316022 [5:00:44<00:00, 17.51it/s]    


⚙️ Corrigiendo columna: txt506 ...


Pandas Apply: 100%|██████████| 316022/316022 [13:04:06<00:00,  6.72it/s]      



✅ Limpieza y corrección lingüística avanzada completada (versión híbrida).


,txt505,txt505b,txt506,texto_final
104908,PROMOTOR DE VENTAS,VENDER OFRECER PLANES DE EQUIPOS,VENTA POR MENOR DE EQUIPOS TELEFONICOS Y PLANES EN LOCAL,promotor de ventas vender ofrecer planes de equipos venta por menor de equipos telefonicos y planes en local
3892,AYUDANTE AGROPECUARIO,PASTEAR JUNTAR AMARRAR OVINO PORCINO DESHIERBAR MAIZ,CULTIVO MAIZ PAPA TRIGO CRIANZA EQUINO OVINO PORCINO,ayudante agropecuario pasear juntar amarrar ovino porcino deshierbar maíz cultivo maíz papa trigo crianza equino ovi...
158463,PRODUCTOR AGROPECUARIO,CORTAR MALEZA ECHAR ABONO ALIMENTAR ANIMALES,CULTIVO DE MAIZ PAPA CRIANZA DE CUY AVES DE CORRAL GANADO PORCINO,productor agropecuario cortar maleza echar abono alimentar animales cultivo de maíz papa crianza de cuy aves de corr...
130769,AYUDANTE PECUARIO,AYUDAR ATENDER AVES DE CORRAL,CRIANZA DE GANADO OVINO AUQUENIDO VACUNO AVES DE CORRAL,ayudante pecuario ayudar atender aves de corral crianza de ganado ovino auquénido vacuno aves de corral
87795,AYUDANTE AGROPECUARIA,DESHIERBAR CULTIVOS DE MAIZ DAR AGUA ALIMENTOS A LOS ANIMALES,CULTIVO MAIZ FRIJOL YUCA CACAO CRIANZA DE VACUNO EQUINO PORCINO AVES D,ayudante agropecuaria deshierbar cultivos de maíz dar agua alimentos a los animales cultivo maíz frijol yuca cacao c...
105899,OPERARIO DE METAL MECANICA,ARMAR PIEZAS DE METAL,TALLER DE METAL MECANICA,operario de metal mecanica armar piezas de metal taller de metal mecanica
23688,AYUDANTE DE PREPARADORA VENDEDORA DE COMIDA,CORTAR PICAR LIMPIAR MEZA DOBLAR LAS SERVILLETA,PREPARACION VENTA DE COMIDA EN RESTAURANT,ayudante de preparadora vendedora de comida cortar picar limpiar meza doblar las servilleta preparación venta de com...
40554,AYUDANTE DE VENDEDOR DE GAS PETRÓLEO GASOLINA,AYUDAR A VENDER GAS PETRÓLEO GASOLINA,VENTA DE GAS PETRÓLEO GASOLINA AL POR MAYOR MENOR EN GRIFO,ayudante de vendedor de gas petróleo gasolina ayudar a vender gas petróleo gasolina venta de gas petróleo gasolina a...
218262,PEON AGRICOLA,CARGAR SANDIA,CULTIVO DE SANDIA,peón agricola cargar sandia cultivo de sandia
277073,PERSONAL DE LIMPIEZA,LIMPIAR BAÑOS,CULTIVO Y EXPORTACIÓN DE UVA ARÁNDANOS FRESCO,personal de limpieza limpiar baños cultivo y exportación de uva arándanos fresco


In [43]:
display(df_sub[["p505r4_label","p506r4_label"]].sample(10, random_state=42))

,p505r4_label,p506r4_label
104908,representantes comerciales,"venta al por menor de pc, unidades perifÃ©ricas, prog informÃ¡ticos y eq de telecomunicaciones en almacenes especial..."
3892,peones de explotaciones agrÃ­colas y ganaderas,cultivo de productos agrÃ­colas en combinaciÃ³n con la crÃ­a de animales (explotaciÃ³n mixta)
158463,"agricultores y trabajadores calificados de cultivos mixtos (agrÃ­colas, pecuarios y forestales)",cultivo de productos agrÃ­colas en combinaciÃ³n con la crÃ­a de animales (explotaciÃ³n mixta)
130769,peones de explotaciones agrÃ­colas y ganaderas,crÃ­a de ovejas y cabras
87795,peones de explotaciones agrÃ­colas y ganaderas,cultivo de productos agrÃ­colas en combinaciÃ³n con la crÃ­a de animales (explotaciÃ³n mixta)
105899,herreros y forjadores,fabricaciÃ³n de productos metÃ¡licos para uso estructural
23688,ayudantes de cocina,restaurantes y servicios mÃ³viles de alimentaciÃ³n
40554,vendedores mayoristas,"venta al por mayor de combustibles sÃ³lidos, lÃ­quidos y gaseosos y productos conexos"
218262,peones de explotaciones agrÃ­colas y ganaderas,"cultivo de vegetales y melones, raÃ­ces y tubÃ©rculos"
277073,"limpiadores y asistentes de oficinas, hoteles y otros establecimientos",cultivo de uvas


In [44]:
import ftfy

df_sub["p505r4_label"] = df_sub["p505r4_label"].astype(str).apply(ftfy.fix_text)
display(df_sub[["p505r4_label"]].sample(5, random_state=42))

,p505r4_label
104908,representantes comerciales
3892,peones de explotaciones agrícolas y ganaderas
158463,"agricultores y trabajadores calificados de cultivos mixtos (agrícolas, pecuarios y forestales)"
130769,peones de explotaciones agrícolas y ganaderas
87795,peones de explotaciones agrícolas y ganaderas


In [45]:
df_sub["p506r4_label"] = df_sub["p506r4_label"].astype(str).apply(ftfy.fix_text)
display(df_sub[["p506r4_label"]].sample(5, random_state=42))

,p506r4_label
104908,"venta al por menor de pc, unidades periféricas, prog informáticos y eq de telecomunicaciones en almacenes especializ..."
3892,cultivo de productos agrícolas en combinación con la cría de animales (explotación mixta)
158463,cultivo de productos agrícolas en combinación con la cría de animales (explotación mixta)
130769,cría de ovejas y cabras
87795,cultivo de productos agrícolas en combinación con la cría de animales (explotación mixta)


In [46]:
display(df_sub[["p505r4_label","p506r4_label"]].sample(10, random_state=42))

,p505r4_label,p506r4_label
104908,representantes comerciales,"venta al por menor de pc, unidades periféricas, prog informáticos y eq de telecomunicaciones en almacenes especializ..."
3892,peones de explotaciones agrícolas y ganaderas,cultivo de productos agrícolas en combinación con la cría de animales (explotación mixta)
158463,"agricultores y trabajadores calificados de cultivos mixtos (agrícolas, pecuarios y forestales)",cultivo de productos agrícolas en combinación con la cría de animales (explotación mixta)
130769,peones de explotaciones agrícolas y ganaderas,cría de ovejas y cabras
87795,peones de explotaciones agrícolas y ganaderas,cultivo de productos agrícolas en combinación con la cría de animales (explotación mixta)
105899,herreros y forjadores,fabricación de productos metálicos para uso estructural
23688,ayudantes de cocina,restaurantes y servicios móviles de alimentación
40554,vendedores mayoristas,"venta al por mayor de combustibles sólidos, líquidos y gaseosos y productos conexos"
218262,peones de explotaciones agrícolas y ganaderas,"cultivo de vegetales y melones, raíces y tubérculos"
277073,"limpiadores y asistentes de oficinas, hoteles y otros establecimientos",cultivo de uvas


In [49]:
display(df_sub[["p301a1_label","p510_label", "p301a_label", "p507_label"]].sample(50, random_state=42))

,p301a1_label,p510_label,p301a_label,p507_label
104908,<NA>,empresa o patrono privado,secundaria completa,empleado
3892,<NA>,<NA>,secundaria incompleta,trabajador familiar no remunerado
158463,<NA>,<NA>,secundaria completa,trabajador independiente
130769,<NA>,<NA>,primaria completa,trabajador familiar no remunerado
87795,<NA>,<NA>,primaria incompleta,trabajador familiar no remunerado
105899,mecanica automotriz,empresa o patrono privado,superior no universitaria completa,obrero
23688,ingeniería ambiental,empresa o patrono privado,superior universitaria incompleta,otro
40554,<NA>,<NA>,secundaria completa,trabajador familiar no remunerado
218262,<NA>,empresa o patrono privado,secundaria completa,obrero
277073,<NA>,empresa o patrono privado,secundaria incompleta,obrero


In [50]:
df_sub["p510_label"] = df_sub["p510_label"].astype(str).apply(ftfy.fix_text)
display(df_sub[["p510_label"]].sample(5, random_state=42))

,p510_label
104908,empresa o patrono privado
3892,<NA>
158463,<NA>
130769,<NA>
87795,<NA>


In [51]:
display(df_sub[["p301a1_label","p510_label", "p301a_label", "p507_label"]].sample(20, random_state=42))

,p301a1_label,p510_label,p301a_label,p507_label
104908,<NA>,empresa o patrono privado,secundaria completa,empleado
3892,<NA>,<NA>,secundaria incompleta,trabajador familiar no remunerado
158463,<NA>,<NA>,secundaria completa,trabajador independiente
130769,<NA>,<NA>,primaria completa,trabajador familiar no remunerado
87795,<NA>,<NA>,primaria incompleta,trabajador familiar no remunerado
105899,mecanica automotriz,empresa o patrono privado,superior no universitaria completa,obrero
23688,ingeniería ambiental,empresa o patrono privado,superior universitaria incompleta,otro
40554,<NA>,<NA>,secundaria completa,trabajador familiar no remunerado
218262,<NA>,empresa o patrono privado,secundaria completa,obrero
277073,<NA>,empresa o patrono privado,secundaria incompleta,obrero


In [52]:
# ============================================================================
# CELDA 12: INSPECTOR DE CORRECCIONES
# ============================================================================
from difflib import get_close_matches

palabra_buscar = "cultibo"  # Cambia la palabra a inspeccionar

# Verificamos si el corrector tiene una sugerencia directa
correccion = spell.correction(palabra_buscar)
candidatos = spell.candidates(palabra_buscar)

print(f"🔍 Análisis para la palabra: '{palabra_buscar}'\n")

if correccion and correccion != palabra_buscar:
    print(f"✅ Sugerencia principal: '{correccion}'")
else:
    print(f"⚠️ No hay corrección segura registrada para '{palabra_buscar}'")

if candidatos:
    print("\n📋 Candidatos similares encontrados por SpellChecker:")
    for cand in sorted(candidatos, key=lambda w: zipf_frequency(w, "es"), reverse=True):
        print(f"  • {cand} (frecuencia: {zipf_frequency(cand, 'es'):.2f})")
else:
    print("❌ No se encontraron candidatos similares en el vocabulario del corrector.")

# Búsqueda adicional en el vocabulario español (wordfreq)
print("\n📚 Palabras del vocabulario español similares:")
vocab_matches = get_close_matches(palabra_buscar, list(spanish_vocab), n=10, cutoff=0.8)
for m in vocab_matches:
    print(f"  • {m} (frecuencia: {zipf_frequency(m, 'es'):.2f})")

if not vocab_matches:
    print("⚠️ No se encontraron coincidencias cercanas en el vocabulario general.")


🔍 Análisis para la palabra: 'cultibo'

✅ Sugerencia principal: 'cultivo'

📋 Candidatos similares encontrados por SpellChecker:
  • cultivo (frecuencia: 4.32)

📚 Palabras del vocabulario español similares:
  • cultivo (frecuencia: 4.32)
  • culto (frecuencia: 4.33)
  • cultivos (frecuencia: 4.12)


In [53]:
import os
print("📂 Ruta actual:", os.getcwd())

📂 Ruta actual: c:\Users\Hugo Jazyel\OneDrive\INEI\PEU-CD\Cursos especializados\Proyecto Integrador


In [54]:
df_sub.to_parquet("BASE_LIMPIA_VF.parquet", index=False)
print("✅ Archivo guardado como BASE_LIMPIA_VF.parquet")

✅ Archivo guardado como BASE_LIMPIA_VF.parquet
